# 4 Prediction of the aggregate demand 

In this section, we will compare the performance of different algorithms in predicting the aggregate demand, separating standard and ToU users. First, we will use feature models and then exponential smoothing, which is a time-series dedicated algorithm.

After adding the external variables (holidays, weather) we will use different algorithms to fit our feature model. Later, we will predict the aggregate demand with exponential smoothing models in R using the "forecast" package.

In general we will train our model with Nov 2011- Nov 2013 data and validate it with Nov 2013 - Feb 2014 data.

We will use the aggregate statistics we computed in Section 2 with Spark.

## 4.1 Preparing the dataset

First, we will prepare our data so it can be use in the different algorithms proposed. We will start including time and weather variables as these are needed for all models and then we will add the engineered features for the feature models.

In [1]:
import pandas as pd
pd.options.display.max_columns = 999
import datetime

In [2]:
df = pd.read_csv('outputs/agg_stats.csv', index_col=0)

In [3]:
df.head()

,DateTime,Tariff,ToU_User,count,sum,min,mean,max,std_dev
0,2012-10-15 21:00:00,Std,0,4210,1270.587000,0.0,0.301802,5.335,0.319130
1,2012-10-21 21:00:00,Std,0,4287,1239.813999,0.0,0.289203,6.095,0.320852
2,2012-10-27 13:00:00,Std,0,4402,1159.695000,0.0,0.263447,3.554,0.336398
3,2012-10-28 17:30:00,Std,0,4402,1665.008001,0.0,0.378239,8.040,0.447315
4,2012-11-04 00:30:00,Std,0,4403,994.769000,0.0,0.225930,6.072,0.424697


We keep only the columns we need: DateTime, Tariff and mean consumption. We are interested in predicting the aggregate demand (i.e. the sum) but as the data count in each Timestamp is different, we will use the mean. However, we cannot take the mean column directly as the value of Tariff is the user group rather than the tariff applied. In 2011, 2012 and 2014 we are interested in the aggregate demand of both groups combined so we will group and calculate the mean with sum and count.

In [4]:
df = df[['DateTime','Tariff','sum','count']]

In [5]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [6]:
type(df['DateTime'][2])

pandas.tslib.Timestamp

In [7]:
df.sample(5)

,DateTime,Tariff,sum,count
54595,2013-01-10 15:00:00,Std,1057.932001,4394
46811,2012-03-26 03:00:00,Std,140.707000,992
11569,2012-01-23 22:00:00,ToU,23.286000,72
40296,2013-06-04 08:30:00,Std,747.785000,4261
47302,2011-12-30 11:30:00,Std,88.511000,352


### 4.1.1 Tariff

On of the main variables to include in our dataset is the price of electricity. We have grouped standard flat rate and dToU users in two different groups but we need the actual values of the tariff. 

For standard users, the price is always equal to 14.228 p/kWh.

For ToU users, the tariff level for each 30-min interval is given in an Excel sheet we downloaded in Section 3.2. The corresponding values for each level are:

-High = 67.20p/kWh

-Normal = 11.76p/kWh

-Low = 3.99p/kWh

In [9]:
tariffs = pd.read_excel('data/Tariffs.xlsx')

In [10]:
tariffs.loc[tariffs['Tariff'] == 'Normal', 'Tariff_value'] = 11.76
tariffs.loc[tariffs['Tariff'] == 'High', 'Tariff_value'] = 67.20
tariffs.loc[tariffs['Tariff'] == 'Low', 'Tariff_value'] = 3.99

In [11]:
tariffs = tariffs.rename(columns = {'TariffDateTime' : 'DateTime'})

In [12]:
tariffs.sample(5)

,DateTime,Tariff,Tariff_value
2505,2013-02-22 04:30:00,Low,3.99
12254,2013-09-13 07:00:00,Normal,11.76
4121,2013-03-27 20:30:00,High,67.20
7229,2013-05-31 14:30:00,Normal,11.76
575,2013-01-12 23:30:00,Normal,11.76


In [13]:
tariffs = tariffs.drop('Tariff', axis=1)

In [14]:
df = df.merge(tariffs, how = 'left', on = 'DateTime')

In [15]:
df['Tariff_value'] = df['Tariff_value'].fillna(14.228) #This is the standard tariff
df.loc[df['User_group'] == 'Std','Tariff_value'] = 14.228

In [16]:
df[['DateTime','User_group','Tariff_value']].sample(5)

,DateTime,User_group,Tariff_value
22792,2013-01-24 06:00:00,Std,14.228
36783,2013-03-25 18:00:00,Std,14.228
66445,2012-04-08 08:00:00,ToU,14.228
7040,2012-09-02 19:00:00,Std,14.228
71395,2013-06-14 19:30:00,ToU,11.760


In [17]:
df = df.groupby(['DateTime', 'Tariff_value']).agg({'sum': 'sum',
                                            'count': 'sum'}).reset_index()

Now we can calculate the mean for each data point.

In [18]:
df['mean'] = df['sum']/df['count']

We also add a new variable "ToU" which indicates whether a consumption data point is for a user in which ToU was applied. As 14.228 p/kWh is the flat rate tariff (and this is not a possible value of ToU tariff), we know that if Tariff_value = 14.228, this is a standard point, and otherwise a ToU point.

In [19]:
df.loc[df['Tariff_value'] == 14.228,'ToU'] = 0
df.loc[df['Tariff_value'] != 14.228,'ToU'] = 1

In [20]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean,ToU
45637,2013-10-01 03:00:00,14.228,406.701000,4154,0.097906,0.0
19865,2013-01-05 15:00:00,14.228,1198.600999,4400,0.272409,0.0
54018,2013-12-27 10:30:00,3.990,262.715000,1041,0.252368,1.0
17903,2012-11-30 09:30:00,14.228,1448.170999,5525,0.262112,0.0
17864,2012-11-29 14:00:00,14.228,1206.686000,5527,0.218326,0.0


### 4.1.2. Bank Holidays

Bank Holidays probably have an impact in household electricity consumption as it was discussed in Section 1. Bank Holidays will be treated separately from Sundays as a binary variable. 

Bank Holidays in England from 2011 to 2014 were downloaded in Section 1 using the Python holidays library (and cross-checked with the official information in www.gov.co.uk) and saved to a csv file.

In [21]:
holidays = pd.Series.from_csv('data/bank_holidays.csv')

In [22]:
holidays = pd.to_datetime(holidays)

In [23]:
holidays.head()

0   2011-01-01
1   2011-01-03
2   2011-04-22
3   2011-04-25
4   2011-04-29
dtype: datetime64[ns]

In [24]:
df['Date'] = df['DateTime'].apply(lambda x: x.date())

In [25]:
df['Date'] = pd.to_datetime(df['Date'])

In [26]:
df['Holiday'] = 0
df.loc[df['Date'].isin(holidays),'Holiday'] = 1

In [27]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean,ToU,Date,Holiday
55691,2014-01-26 17:30:00,14.228,1926.090,5067,0.380124,0.0,2014-01-26,0
46793,2013-10-13 04:00:00,14.228,445.236,4150,0.107286,0.0,2013-10-13,0
15681,2012-10-15 02:30:00,14.228,576.639,5260,0.109627,0.0,2012-10-15,0
2454,2012-01-13 12:00:00,14.228,111.795,434,0.257592,0.0,2012-01-13,0
25795,2013-03-08 09:30:00,14.228,1002.496,4326,0.231737,0.0,2013-03-08,0


In [28]:
df[df['Holiday'] == 1].sample(5)

,DateTime,Tariff_value,sum,count,mean,ToU,Date,Holiday
53784,2013-12-25 00:00:00,11.760,210.494000,1042,0.202010,1.0,2013-12-25,1
19169,2012-12-26 18:30:00,14.228,1937.954999,5508,0.351844,0.0,2012-12-26,1
53851,2013-12-25 16:30:00,14.228,1373.703999,4057,0.338601,0.0,2013-12-25,1
19465,2013-01-01 11:00:00,14.228,1069.331000,4398,0.243140,0.0,2013-01-01,1
33433,2013-05-27 00:00:00,14.228,663.014000,4266,0.155418,0.0,2013-05-27,1


### 4.1.3. Weather variables

Now we have to add the weather variables, which will be used in any model. Weather variables were discussed, downloaded and treated in Section 1.

In [29]:
weather = pd.read_csv('data/weather_no_na.csv')

In [30]:
weather.shape

(20446, 13)

We will use the following variables:

-Temperature (ºC)

-Relative Humidity (%)

-Cloud cover (%)

-Atmospheric Pressure (mbar)

For a discussion on the reasons why these variables have been choosen and how NA values have been filled refer to Section 1.

In [31]:
weather.columns

Index(['time', 'apparentTemperature', 'cloudCover', 'dewPoint', 'humidity',
       'icon', 'precipType', 'pressure', 'summary', 'temperature',
       'visibility', 'windBearing', 'windSpeed'],
      dtype='object')

In [32]:
weather = weather[['time','temperature','humidity','cloudCover','pressure']]\
    .rename(columns = {'time' : 'DateTime'})

In [33]:
weather['DateTime'] = pd.to_datetime(weather['DateTime'])

In [34]:
weather.head()

,DateTime,temperature,humidity,cloudCover,pressure
0,2011-11-01 00:00:00,13.54,0.87,0.27,1008.01
1,2011-11-01 01:00:00,12.74,0.93,0.32,1007.76
2,2011-11-01 02:00:00,13.68,0.91,0.25,1006.97
3,2011-11-01 03:00:00,14.18,0.88,0.43,1006.40
4,2011-11-01 04:00:00,14.20,0.90,0.38,1006.05


The frequency of the weather data is of 1 hour while that of the consumption data is 30 minutes, thus we need to resample it. We will take the last valid observation (e.g. 2012-01-01 10:30 will have the same data as 2012-01-01 10:00) => method = 'ffill'.

In [35]:
weather = weather.set_index('DateTime').resample('30min').fillna(method = 'ffill')\
    .reset_index()

In [36]:
weather.head()

,DateTime,temperature,humidity,cloudCover,pressure
0,2011-11-01 00:00:00,13.54,0.87,0.27,1008.01
1,2011-11-01 00:30:00,13.54,0.87,0.27,1008.01
2,2011-11-01 01:00:00,12.74,0.93,0.32,1007.76
3,2011-11-01 01:30:00,12.74,0.93,0.32,1007.76
4,2011-11-01 02:00:00,13.68,0.91,0.25,1006.97


In [37]:
df = df.merge(weather, how = 'left', on = 'DateTime')

In [38]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure
29269,2013-04-13 15:00:00,14.228,1088.699000,4291,0.253717,0.0,2013-04-13,0,10.31,0.90,0.82,1010.77
20162,2013-01-08 17:30:00,11.760,374.946999,1111,0.337486,1.0,2013-01-08,0,10.70,0.85,0.75,1023.84
43736,2013-09-11 08:00:00,11.760,180.339000,1055,0.170937,1.0,2013-09-11,0,13.54,0.73,0.75,1021.87
34903,2013-06-11 07:30:00,14.228,824.260000,4249,0.193989,0.0,2013-06-11,0,12.11,0.80,1.00,1013.63
39153,2013-07-25 14:00:00,14.228,689.957000,4214,0.163730,0.0,2013-07-25,0,24.32,0.59,0.64,1012.31


Now that we have added all external variables our models need, we can save it to a csv file. 

In [39]:
df.to_csv('outputs/tseries_model.csv')

We will use this model as is in time series modelling. For feature models we need to add the features.

## 4.1.4. Feature engineering

Feature models are an alternative for time series forecasting. They consist in applying conventional machine learning models to variables constructed from the data (e.g. mean consumption in the previous day). These are popular in the literature for electrical demand prediction [2], [4].

We will derive the following features:

-Mean aggregate demand per hour in the previous day (1 variable).

-Aggregate demand in the previous day at the same time and at the previous 3 time steps (4 variables).

-Mean value of the aggregate demand of the previous 3 days at the same time and at the previous 3 time steps (4 variables).

-Mean value of the aggregate demand on the same day of week of the previous 3 weeks at the same time and at the previous 3 time steps (4 variables).

In total we have 13 derived features.

Furthermore, we will include weather and time variables in the model:

-Day of week (7 levels)

-Holiday (binary)

-Temperature (ºC, continuous)

-Relative Humidity (%, continuous)

-Cloud cover (%, continuous)

-Atmospheric Pressure (mbar, continuous)

These variables have been calculated in Section 1 and stored in 'weather_no_na.csv'.

And the last variable to be included in the model:

-Tariff (p/kWh)

Thus, in total we will have 22 variables in our models.


First, we will build the time variables.

Now it is easier to derive the consumption features.

#### Adding time variables

Let us add the time variables to be used in lieu of DateTime in our prediction model: time, day of week, month, year.

We will not use the day of the month as it would introduce a categorical variable with 31 levels and it does not seem relevant in the exploratory data analysis in Tableau.

In [40]:
df['DoW'] = df['DateTime'].apply(lambda x: x.strftime("%A"))

In [41]:
df['Time'] = df['DateTime'].apply(lambda x: x.time())

In [42]:
df['Month']= df['DateTime'].apply(lambda x: x.strftime("%B"))

In [43]:
df['Year']= df['DateTime'].apply(lambda x: x.year)

In [44]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year
1161,2011-12-17 13:30:00,14.228,91.504,279,0.327971,0.0,2011-12-17,0,5.02,0.84,0.22,1008.78,Saturday,13:30:00,December,2011
50814,2013-11-24 01:30:00,11.760,139.300,1045,0.133301,1.0,2013-11-24,0,6.21,0.79,0.07,1026.92,Sunday,01:30:00,November,2013
18278,2012-12-08 05:00:00,14.228,769.883,5526,0.139320,0.0,2012-12-08,0,4.62,0.81,1.00,1020.83,Saturday,05:00:00,December,2012
17596,2012-11-24 00:00:00,14.228,1106.375,5527,0.200176,0.0,2012-11-24,0,4.92,0.99,0.09,1016.48,Saturday,00:00:00,November,2012
16135,2012-10-24 13:30:00,14.228,1018.698,5447,0.187020,0.0,2012-10-24,0,15.74,0.85,0.69,1018.91,Wednesday,13:30:00,October,2012


###### Mean aggregate demand per hour in the previous day (1 variable).

First, we will calculate the daily mean aggregating by 'Date'. Then we will add one day to the 'Date' variable of our new DataFrame and join it with the main DataFrame df.

In [45]:
daily_mean = df.groupby(['Date', 'ToU']).agg({'mean': 'mean'}).reset_index()

In [46]:
daily_mean['Date'] = pd.to_datetime(daily_mean['Date'])

In [47]:
type(daily_mean['Date'][2])

pandas.tslib.Timestamp

In [48]:
daily_mean.sample(5)

,Date,ToU,mean
654,2013-05-05,1.0,0.172332
640,2013-04-28,1.0,0.191478
1068,2013-11-28,1.0,0.214057
629,2013-04-23,0.0,0.183648
908,2013-09-09,1.0,0.169568


In [49]:
daily_mean['Date'] = daily_mean['Date'] + datetime.timedelta(days=1)

In [50]:
df = df.merge(daily_mean, how = 'left', on = ['Date','ToU'])

In [51]:
df = df.rename(columns = {'mean_x' : 'mean_cons',
          'mean_y' : 'mean_prev_day'})

In [52]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean_cons,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year,mean_prev_day
55862,2014-01-30 07:00:00,14.228,1030.818,5058,0.203800,0.0,2014-01-30,0,3.73,0.92,1.00,1000.72,Thursday,07:00:00,January,2014,0.246226
44040,2013-09-14 12:00:00,11.760,223.519,1057,0.211465,1.0,2013-09-14,0,12.66,0.88,1.00,1016.24,Saturday,12:00:00,September,2013,0.175593
34028,2013-06-02 05:00:00,11.760,103.525,1089,0.095064,1.0,2013-06-02,0,9.98,0.79,0.06,1027.83,Sunday,05:00:00,June,2013,0.164209
24802,2013-02-26 01:30:00,11.760,158.437,1106,0.143252,1.0,2013-02-26,0,3.01,0.90,0.90,1029.08,Tuesday,01:30:00,February,2013,0.253990
2912,2012-01-23 01:00:00,14.228,122.919,493,0.249329,0.0,2012-01-23,0,6.81,0.72,0.32,1018.08,Monday,01:00:00,January,2012,0.285867


###### Aggregate demand in the previous day at the same time and at the previous 3 time steps (4 variables).

In [53]:
df.head()

,DateTime,Tariff_value,sum,count,mean_cons,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year,mean_prev_day
0,2011-11-23 09:00:00,14.228,0.569,2,0.284500,0.0,2011-11-23,0,4.84,0.99,0.32,1027.34,Wednesday,09:00:00,November,2011,NaN
1,2011-11-23 09:30:00,14.228,0.561,2,0.280500,0.0,2011-11-23,0,4.84,0.99,0.32,1027.34,Wednesday,09:30:00,November,2011,NaN
2,2011-11-23 10:00:00,14.228,0.920,6,0.153333,0.0,2011-11-23,0,5.69,0.98,0.56,1027.72,Wednesday,10:00:00,November,2011,NaN
3,2011-11-23 10:30:00,14.228,0.588,6,0.098000,0.0,2011-11-23,0,5.69,0.98,0.56,1027.72,Wednesday,10:30:00,November,2011,NaN
4,2011-11-23 11:00:00,14.228,0.772,7,0.110286,0.0,2011-11-23,0,7.66,0.88,0.32,1027.59,Wednesday,11:00:00,November,2011,NaN


In [54]:
def prev_ts(df_in, initial, final, step):
    #Both initial and final are included in the loop
    df_out = df_in[['DateTime', 'ToU']].copy()
    for i in range(initial, final + 1, step):
        aux = df_in[['DateTime', 'ToU', 'mean_cons']].copy()
        aux['DateTime'] = aux['DateTime'] + datetime.timedelta(minutes = i)
        df_out = df_out.\
            merge(aux, how = 'left', on = ['DateTime','ToU'], suffixes = ('','_-%d' %(i)))
    return(df_out)

In [55]:
tmp = prev_ts(df, 22*60+30, 24*60, 30)
df = df.merge(tmp[['DateTime','ToU','mean_cons_-1380', 'mean_cons_-1410', 'mean_cons_-1440']],
         on = ['DateTime','ToU'], suffixes = ('',''))

In [56]:
df.sample(5)

,DateTime,Tariff_value,sum,count,mean_cons,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year,mean_prev_day,mean_cons_-1380,mean_cons_-1410,mean_cons_-1440
49516,2013-11-10 13:00:00,11.760,291.675000,1050,0.277786,1.0,2013-11-10,0,9.17,0.56,0.31,1018.63,Sunday,13:00:00,November,2013,0.216511,0.249241,0.258339,0.270548
10267,2012-06-24 07:30:00,14.228,700.699000,4173,0.167913,0.0,2012-06-24,0,13.89,0.95,0.89,1008.89,Sunday,07:30:00,June,2012,0.179633,0.197776,0.184852,0.175956
7440,2012-04-26 10:00:00,14.228,381.664000,1929,0.197856,0.0,2012-04-26,0,11.60,0.77,0.49,993.71,Thursday,10:00:00,April,2012,0.221164,0.218806,0.222292,0.224264
33780,2013-05-30 15:00:00,11.760,197.468000,1089,0.181330,1.0,2013-05-30,0,14.03,0.79,0.81,1011.53,Thursday,15:00:00,May,2013,0.182254,0.212861,0.202480,0.192560
31301,2013-05-04 19:00:00,14.228,1087.188999,4273,0.254432,0.0,2013-05-04,0,14.07,0.50,0.33,1018.25,Saturday,19:00:00,May,2013,0.177542,0.257257,0.244308,0.233400


###### Mean value of the aggregate demand of the previous 3 days at the same time and at the previous 3 time steps (4 variables).

In [57]:
aux2 = df[['DateTime', 'ToU']].copy()
means_3d = pd.DataFrame(index=df.index)
for h in range(0,4):
    for d in range(1,4):
        aux = df[['DateTime', 'ToU', 'mean_cons']].copy()
        aux['DateTime'] = aux['DateTime'] + datetime.timedelta(minutes = d*24*60-h*30)
        aux2 = aux2.\
            merge(aux, how = 'left', on = ['DateTime','ToU'])
    means_3d = means_3d.join(aux2.mean(axis=1).rename('mean_last3d_-%d' %(h*30)))

In [58]:
df = df.join(means_3d)

In [59]:
df.sample(3)

,DateTime,Tariff_value,sum,count,mean_cons,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year,mean_prev_day,mean_cons_-1380,mean_cons_-1410,mean_cons_-1440,mean_last3d_-0,mean_last3d_-30,mean_last3d_-60,mean_last3d_-90
25051,2013-02-28 15:30:00,14.228,1000.229,4333,0.230840,0.0,2013-02-28,0,7.34,0.69,0.68,1028.05,Thursday,15:30:00,February,2013,0.258202,0.254149,0.241016,0.227001,0.190855,0.224592,0.243375,0.258977
24096,2013-02-18 17:00:00,11.760,273.904,1103,0.248326,1.0,2013-02-18,0,5.78,0.73,0.43,1019.80,Monday,17:00:00,February,2013,0.236516,0.358004,0.326865,0.282129,0.455238,0.395348,0.382492,0.380767
14430,2012-09-19 01:00:00,14.228,548.903,4846,0.113269,0.0,2012-09-19,0,8.73,0.77,0.00,1020.20,Wednesday,01:00:00,September,2012,0.177089,0.097233,0.104080,0.112865,0.087024,0.094739,0.096186,0.096378


###### Mean value of the aggregate demand on the same day of week of the previous 3 weeks at the same time and at the previous 3 time steps (4 variables).

In [60]:
aux2 = df[['DateTime', 'ToU']].copy()
means_3w = pd.DataFrame(index=df.index)
for h in range(0,4):
    for w in range(1,4):
        aux = df[['DateTime', 'ToU', 'mean_cons']].copy()
        aux['DateTime'] = aux['DateTime'] + datetime.timedelta(minutes = w*7*24*60-h*30)
        aux2 = aux2.\
            merge(aux, how = 'left', on = ['DateTime','ToU'])
    means_3w = means_3w.join(aux2.mean(axis=1).rename('mean_last3w_-%d' %(h*30)))

In [61]:
df = df.join(means_3w)

In [62]:
df.sample(3)

,DateTime,Tariff_value,sum,count,mean_cons,ToU,Date,Holiday,temperature,humidity,cloudCover,pressure,DoW,Time,Month,Year,mean_prev_day,mean_cons_-1380,mean_cons_-1410,mean_cons_-1440,mean_last3d_-0,mean_last3d_-30,mean_last3d_-60,mean_last3d_-90,mean_last3w_-0,mean_last3w_-30,mean_last3w_-60,mean_last3w_-90
12186,2012-08-03 07:00:00,14.228,755.137,4758,0.158709,0.0,2012-08-03,0,15.58,0.88,0.40,1013.10,Friday,07:00:00,August,2012,0.165985,0.174426,0.163972,0.163186,0.124002,0.142987,0.152077,0.157382,0.132612,0.153827,0.161793,0.165229
15585,2012-10-13 02:30:00,14.228,585.085,5258,0.111275,0.0,2012-10-13,0,7.89,0.89,0.33,1006.72,Saturday,02:30:00,October,2012,0.200887,0.105021,0.105890,0.107292,0.080645,0.091444,0.095743,0.098331,0.078329,0.088411,0.092363,0.094512
4837,2012-03-03 03:30:00,14.228,149.032,863,0.172691,0.0,2012-03-03,0,7.56,0.97,0.73,1018.72,Saturday,03:30:00,March,2012,0.248041,0.152221,0.155141,0.161457,0.120545,0.133285,0.137457,0.139172,0.149946,0.164244,0.166726,0.169035


Note that pandas.dataframe.mean skips NaN values by default. Therefore, if some values are missing, the mean is still calculated with the remaining ones. Nonetheless, in the first 3 weeks, none of the values needed to calculate some features will exist so there will be missing values. These values will be dropped from the model. This is not needed in dedicated time series algorithms.

In [63]:
df = df.dropna().reset_index(drop=True)

We will split the dataset into train and test samples: from the dataset start to 2013-09-31 will be included in the train set and from 2013-10-01 until the end of the time series in the test set. This means we will have 9 months of ToU in train and 3 in test. The limitation is not all months will be included in the test set and we will not have one whole year of ToU data in the train set.

In [64]:
df.loc[df['DateTime'] < '2013-10-01 00:00:00','Set'] = 'Train'
df.loc[df['DateTime'] >= '2013-10-01 00:00:00','Set'] = 'Test'

Time can be encoded as a categorical variable or as a continuous numerical variable.

Looking at the exploratory data analysis in Tableau, aggregating data by time, we can distinguish a plateau between 8:00 and 15:59 during weekdays, an increasing trend up to 19:00, then decreasing to 2:00 and a plateau between 2:00 and 5:59 before increasing to 8:00. During weekends, the midday plateau is not so constant and seems to start at 9:00.

Considering time with 30-min frequency would result in a categorical variable with 48 levels. This resolution is probably to large for our purpose taking into account that we will derive features based on consumption in previous time steps and at the same time on previous comparable days, as we will see later. 

Considering time as a continuous numerical variable is probably a better approach, but from the exploratory data analysis we now its relationship with the target will be strongly non-linear.

Therefore, we propose to use it as a numerical variable in the interval [0,24) in models which can grasp non-linearities and to drop it in linear models at is has been used to derived the features.

We will also remove the year as we only have 2 complete years (2012 and 2013) and part of our test data is in 2014, which is absent from the train set.

In [65]:
def time2float(time):
    return time.hour + time.minute / 60.0

In [66]:
df['Time'] = df['Time'].apply(lambda t: time2float(t))

Now that we have built all features we can save this model to a csv file for use in R.

In [67]:
df.to_csv('outputs/features_model.csv', index = False)